# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [ ]:
%pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.8 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [ ]:
def generate(nlen=2048):
    while True:
        p = cu.getPrime(nlen//2)
        q = cu.getPrime(nlen//2)
        # cond 0!
        if p * q < pow(2, 2047):
            continue
        # cond 5
        if abs(p - q) <= pow(2, nlen//2 - 100):
            continue
        # cond 3
        if pow(2, (nlen-1) // 2) >= p or pow(2, nlen//2) <= p:
            continue
        # cond 4
        if pow(2, (nlen-1) // 2) >= q or pow(2, nlen//2) <= q:
            continue
        f = (p - 1) * (q - 1)
        # cond 1
        first_e = cu.getRandomRange(pow(2, 16), pow(2, 256))
        e = first_e + 1 # we want 65537 not 65536
        while e != first_e:
            # catching too big e
            if e >= min(pow(2, 256), f) or (math.lcm(p-1, q-1) < e * pow(2, nlen//2)):
                e = 65536 # there is +1 at the end of while
            if math.gcd(f, e) == 1:
                d = cu.inverse(e, f)
                n = p * q
                return (n, e, d)
            e += 1
    
   
n, e, d = generate()


## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [ ]:
def to_bytes(n):
    return n.to_bytes((n.bit_length() + 7) // 8, 'big')

In [ ]:
M = "Ala ma kota"

M_encoded = int.from_bytes(M.encode('utf-8'), 'big')
C = pow(M_encoded, e, n)

M_decoded = pow(C, d, n)
print(to_bytes(M_decoded).decode('utf-8'))

Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [ ]:
from hashlib import sha256

def sign(m, d, n):
    sha = sha256()
    sha.update(m)
    h = cu.bytes_to_long(sha.digest())
    sig = pow(h, d, n)
    return m, sig

def verify(m, sig, e, n):
    sha = sha256()
    sha.update(m)
    h = cu.bytes_to_long(sha.digest())
    ver = pow(sig, e, n)
    return ver == h


m = bytes.fromhex("010203040506")
m, sig = sign(m, d, n)
print(verify(m, sig, e, n))
print(verify(bytes.fromhex("01"), sig, e, n))

True
False


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [ ]:
def mgf1(m, length, h_func=lambda x: sha256(x).digest(), hlen=32):
    output = bytes()
    for i in range(0, math.ceil(length / hlen)):
        i_bytes = i.to_bytes(4, 'big')
        tmp = h_func(m + i_bytes)
        output += tmp
    return output[:length]

In [ ]:
res= mgf1(bytes.fromhex("010203"), 65)
print(res.hex())
print(len(res))

33039fca04699df0cbf328f37d8fd61eeac2612d2d2cddb89d4c6fc4a4a1a3de6d0d07a7d8f116124cf7dc8817849be15168935f3a61991db4effbc3e7bdb8c685
65


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [ ]:
def xor(bytes1, bytes2):
    if len(bytes1) != len(bytes2):
        raise Exception("XOR for two byte arrays of different size")
    return bytes(a ^ b for a, b in zip(bytes1, bytes2))

In [ ]:
from random import randbytes

def oaep_encoding(k, mlen, m, h_func=lambda x: sha256(x).digest(), hlen=32, mgf1=mgf1):
    if mlen > k - 2 * hlen - 2:
        raise Exception("Error")
    l_hash = h_func(b"")
    PS = bytes.fromhex("00" * (k - mlen - 2 * hlen - 2))
    DB = l_hash + PS + bytes.fromhex("01") + m
    print(">>", len(DB), k - hlen - 1)
    seed = randbytes(hlen)
    db_mask = mgf1(seed, k - hlen - 1)
    masked_DB = xor(DB, db_mask)
    seed_mask = mgf1(masked_DB, hlen)
    masked_seed = xor(seed, seed_mask)
    EM = bytes.fromhex("00") + masked_seed + masked_DB
    print(">>", len(EM), k)
    return EM


def oaep_decoding(k, mlen, EM, h_func=lambda x: sha256(x).digest(), hlen=32, mgf1=mgf1):
    Y = EM[0]
    masked_seed = EM[1: 1 + hlen]
    masked_DB = EM[1 + hlen:]
    l_hash = h_func(b"")
    seed_mask = mgf1(masked_DB, hlen)
    seed = xor(masked_seed, seed_mask)
    DB_mask = mgf1(seed, k - hlen - 1)
    DB = xor(masked_DB, DB_mask)
    l_hash_p = DB[:hlen]
    PS = DB[hlen: hlen + k - mlen - 2 * hlen - 2]
    DB = DB[hlen + k - mlen - 2 * hlen - 2: ]
    Y = DB[0]
    M = DB[1:]
    if Y != 1 or l_hash != l_hash_p:
        raise Exception("Error")
    return M

def rsa_oaep_encode(M, mlen, nlen=256):
    M = oaep_encoding(nlen, mlen, M)
    M_encoded = int.from_bytes(M, 'big')
    C = pow(M_encoded, e, n)
    return C

def rsa_oaep_decode(C, mlen, nlen=256):
    M_decoded = pow(C, d, n).to_bytes(nlen, 'big')
    M = oaep_decoding(nlen, mlen, M_decoded)
    return M

In [ ]:
M = "Ala ma kota"
M_bytes = M.encode('utf-8')
print(M_bytes.hex())
print(oaep_encoding(256, len(M_bytes), M_bytes).hex())
C = rsa_oaep_encode(M_bytes, len(M_bytes))
M = rsa_oaep_decode(C, len(M_bytes)).decode('utf-8')
print(M)

416c61206d61206b6f7461
>> 223 223
>> 256 256
001f47288761845de153348f3cbac0fa86a261eda650f405a005bb6eb9bc10f14c807b2b3fddfb495edcb033bf143fbca42adf7996baf0eeb66c22c73fbea04295621d7fb9c1f60b2b6b67505518e4b3c9f17b02ff2f98c2c5833b712fec86b4d80d1577cf9d390a8f3ed34279ae9d2cebd2b5584d12bf3cd718fd3eef55738ffae8152022c0568e5f0818d4d5217098de87ff6a67f554540991c6c487464d488e8bc8379dd3b84394352123b210dcd570eee5a9e9646702e65bed4c740b6810f7465895a6401b7586feafdb47a701d38229de3b8f795c5d5e7e0edd05e281085ecf7fc9c00f4a4f489ff89f5e9c3fd5d718860ccac96164b583bda92283f5cc
>> 223 223
>> 256 256
Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [ ]:
def erease_first_bits(src, bits):
    l = len(src)
    mask = (1 << (len(src)*8 - bits)) ^ (pow(2,  (len(src)*8 - bits) + 1) - 1)
    src = int.from_bytes(src, 'big')
    return (src & mask).to_bytes(l, 'big')

def are_first_bits_zero(src, bits):
    return int.from_bytes(src, 'big') < pow(2, len(src)*8 - bits)

In [ ]:
x = erease_first_bits(bytes.fromhex("FF"*3), 13)
x.hex()

'0007ff'

In [ ]:
def emsa_encoding(M, emlen=256, h_func=lambda x: sha256(x).digest(), hlen=32, embits=2047, slen=32, mgf1=mgf1):
    M_hash = h_func(M)
    if emlen < hlen + slen + 2:
        raise Exception("Error")
    salt = randbytes(slen)
    M_p = bytes.fromhex("00" * 8) + M_hash + salt
    print(len(M_p), 8 + hlen + slen)
    M_hash_p = h_func(M_p)
    PS = bytes.fromhex("00" * (emlen - slen - hlen - 2))
    DB = PS + bytes.fromhex("01") + salt
    DB_mask = mgf1(M_hash_p, emlen - hlen - 1)
    masked_DB = xor(DB, DB_mask)
    masked_DB = erease_first_bits(masked_DB, 8 * emlen - embits)
    return masked_DB + M_hash_p + bytes.fromhex("bc")


def emsa_decoding(EM, M, emlen=256, h_func=lambda x: sha256(x).digest(), hlen=32, embits=2047, slen=32, mgf1=mgf1):
    M_hash = h_func(M)
    if emlen < hlen + slen + 2:
        raise Exception("Error")
    if EM[-1] != 0xbc:
        raise Exception("Error")
    masked_DB = EM[:emlen - hlen - 1]
    M_hash_p = EM[emlen - hlen - 1: emlen - 1]
    if not are_first_bits_zero(masked_DB,  8*emlen - embits):
        raise Exception("Error")
    DB_mask = mgf1(M_hash_p, emlen - hlen - 1)
    DB = xor(masked_DB, DB_mask)
    DB = erease_first_bits(DB, 8 * emlen - embits)
    if (not are_first_bits_zero(DB, 8*(emlen - hlen - slen - 2))) or DB[emlen - hlen - slen - 2] != 1: # -2 as we have 0 based indexing
        raise Exception("Error")
    salt = DB[-slen:]
    print(len(M_hash), len(salt))
    M_p = bytes.fromhex("00" * 8) + M_hash + salt
    print(len(M_p), 8 + hlen + slen)
    M_hash_b = h_func(M_p)
    if M_hash_p != M_hash_b:
        raise Exception("Error")
    print("OK")

In [ ]:
M = "Ala ma podpis".encode('utf-8')
M2 = "Alicja ma podpis".encode('utf-8')
EM = emsa_encoding(M)
emsa_decoding(EM, M)

72 72
32 32
72 72
OK


In [ ]:
emsa_decoding(EM, M2)

32 32
72 72


Exception: ignored

In [ ]:
def sign_emsa(m, d, n):
    encoded = emsa_encoding(m)
    h = cu.bytes_to_long(encoded)
    sig = pow(h, d, n)
    return m, sig

def verify_emsa(m, sig, e, n):
    ver = pow(sig, e, n)
    emsa_decoding(to_bytes(ver), m)
    return True


m = bytes.fromhex("010203040506")
m, sig = sign_emsa(m, d, n)
print(verify_emsa(m, sig, e, n))

72 72
32 32
72 72
OK
True


In [ ]:
print(verify_emsa(bytes.fromhex("01"), sig, e, n))

32 32
72 72


Exception: ignored

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [ ]:
from Crypto.Cipher import AES

In [ ]:
def sender():
    rand = randbytes(320)
    key = sha256(rand).digest()
    ciphered_key = rsa_oaep_encode(key, 32)
    return key, ciphered_key

def receiver(ciphered_key):
    key = rsa_oaep_decode(ciphered_key, 32)
    return key

In [ ]:
key, ciphered_key = sender()
print(key.hex(), ciphered_key)
key2 = receiver(ciphered_key)
print(key2.hex())

>> 223 223
>> 256 256
71175a1a63cac59bd414e2469f6ded47e91a9a76416b9205d78319b211ea65e9 18325184808349368487722190906827561848194443118881294353297992328537098612979229674973675963242008720752244426595953312969093885631269777978730328582130720019048151681947163902696435399026818360009551839837743692886866906539047183078176736975829201097555332527264761903668276329469661055791400112068672090175088309340790056365495690022625672939746103414375808721208695323392655562263539478140311609532576783742324505528219442480462483871658445086616411026560422257193677222038582351629989444123339245711166359213664417258926660761481336835585162503367408964143907585802628723213716589133085277498392656893133549626833
71175a1a63cac59bd414e2469f6ded47e91a9a76416b9205d78319b211ea65e9


In [ ]:
ciphertext = AES.new(key, AES.MODE_ECB).encrypt(b"Ala ma kota Toma")
print(ciphertext.hex())
print(AES.new(key2, AES.MODE_ECB).decrypt(ciphertext))

7985a0a17ce4745a66b875d1882a4639
b'Ala ma kota Toma'
